In [ ]:
from google.colab import files
import sqlite3
import pandas as pd

In [ ]:
#Cargar excels
df1=pd.read_csv("/content/Accounts.csv")
df2=pd.read_csv("/content/Forecasts.csv")
df3=pd.read_csv("/content/Sales.csv")

In [26]:
#Creamos nuestra base de datos
conn=sqlite3.connect("mi_base_de_datos.db")
cursor=conn.cursor()

df1.to_sql("ACCOUNTS", conn, if_exists="replace", index=False)
df2.to_sql("FORECASTS", conn, if_exists="replace", index=False)
df3.to_sql("SALES", conn, if_exists="replace", index=False)

conn.close()


1.Análisis de Ventas y Beneficio por Categoría de Producto para Adabs Entertainment en 2020
Enunciado: Realiza un análisis de las ventas y el beneficio total desglosado por categoría de producto exclusivamente para la cuenta Adabs Entertainment durante el año 2020.

Calcula los siguientes sumatorios:

● Ventas

    ○ Mantenimiento

    ○ Producto

    ○ Partes

    ○ Soporte

    ○ Totales

● Número de unidades vendidas.

● Beneficio total.

In [28]:
#1. Análisis de Ventas y Beneficio por Categoría de Producto para Adabs Entertainment en 2020

conn=sqlite3.connect("mi_base_de_datos.db")
cursor=conn.cursor()

# Definir tu consulta SQL
query = """
    SELECT
        Category AS CategoriaProducto,
        SUM(Maintenance) AS VentasMantenimiento,
        SUM(Product) AS VentasProducto,
        SUM(Parts) AS VentasPartes,
        SUM(Support) AS VentasSoporte,
        SUM(UNITS_SOLD) AS UnidadesVendidasTotales,
        SUM(Total) AS VentasTotales,
        SUM(Profit) AS BeneficioTotal
    FROM
        SALES
    WHERE
        Account = 'Adabs Entertainment'
        AND Year = 2020
    GROUP BY
        Category;
"""
# Ejecutar la consulta

df=df = pd.read_sql_query(query, conn)
df


,CategoriaProducto,VentasMantenimiento,VentasProducto,VentasPartes,VentasSoporte,UnidadesVendidasTotales,VentasTotales,BeneficioTotal
0,Chairs,400000.0,1235672.0,200000.0,0.0,3530.49,1835672.0,605772.0
1,Electronics,250000.0,1250000.0,100000.0,500000.0,3571.43,2100000.0,756000.0


Comparación de Ventas, Unidades Vendidas y Beneficio entre Países en las
Regiones APAC y EMEA

Enunciado: Compara el rendimiento de ventas entre los diferentes países
pertenecientes a las regiones APAC y EMEA. Para obtener los siguientes indicadores:

● Ingreso promedio

● Número de unidades vendidas promedio.

● Beneficio promedio.

In [ ]:
#2.Comparación de Ventas, Unidades Vendidas y Beneficio entre Países en las Regiones APAC y EMEA

query="""
    SELECT
    a.Region,
    a.Country,
    AVG(v.Total ) AS IngresoPromedio,
    AVG(v.Units_Sold) AS UnidadesVendidasPromedio,
    AVG(v.Profit ) AS BeneficioPromedio
FROM
    SALES v /* Tabla principal de ventas */
JOIN
    ACCOUNTS a /* Join con la tabla de cuentas */
ON
    v.Account = a.Account   /* Condición de join usando el campo Account */

WHERE
    a.Region IN ('APAC', 'EMEA')
GROUP BY
    a.Region,
    a.Country
ORDER BY
    a.Region,
    a.Country;
"""

df=pd.read_sql_query(query,conn)
df

,REGION,COUNTRY,IngresoPromedio,UnidadesVendidasPromedio,BeneficioPromedio
0,APAC,Australia,3.266029e+06,6397.833333,1.237014e+06
1,APAC,Japan,1.149073e+06,1989.610000,4.061040e+05
2,APAC,Korea,1.612244e+06,3134.847500,5.970032e+05
3,APAC,Singapore,3.415680e+06,6172.955000,1.150704e+06
4,EMEA,Austria,1.544030e+06,3189.298333,5.377263e+05
5,EMEA,France,1.115716e+06,2267.335714,3.889768e+05
6,EMEA,Germany,1.501904e+06,2996.234545,5.121583e+05
7,EMEA,Italy,9.218675e+05,1859.005000,3.068718e+05
8,EMEA,Portugal,1.377733e+06,3077.504000,5.114416e+05
9,EMEA,Spain,1.215924e+06,2644.576000,4.065416e+05


3 Análisis del Beneficio Total por Industria: Estudio de Clientes en Etapa de
Compromiso

Enunciado: Calcula el beneficio total para cada industria y clasifícalo como "Alto" si
supera los 1,000,000 o como "Normal" si no lo hace. Realiza este cálculo
considerando únicamente las filas correspondientes a las cuentas clasificadas como
Commit (acuerdos cercanos al cierre) y cuyo pronóstico total de beneficios sea superior
a $500,000.

In [ ]:
#3.Análisis del Beneficio Total por Industria: Estudio de Clientes en Etapa de Compromiso

query="""
    WITH industry_profits AS (
  SELECT
    a.industry,
    SUM(s.profit) AS total_profit,
    CASE
      WHEN SUM(s.profit) > 1000000 THEN 'Alto' /* Si el beneficio es mayor a 1M, categoriza como 'Alto' */
      ELSE 'Normal' /* Si no, categoriza como 'Normal' */
    END AS profit_category
  FROM
    sales AS s
    JOIN accounts AS a ON s."ACCOUNT" = a."ACCOUNT" /* Une ventas con cuentas usando el campo ACCOUNT */
  WHERE
    s."ACCOUNT" IN (
      SELECT /* Subconsulta para filtrar cuentas con ventas totales superiores a 500K */
        "ACCOUNT"
      FROM
        sales
      GROUP BY
        "ACCOUNT"
      HAVING
        SUM(total) > 500000
    )
  GROUP BY
    a.industry
)
SELECT
  industry,
  total_profit,
  profit_category
FROM
  industry_profits
ORDER BY
  total_profit DESC;
"""
df=pd.read_sql_query(query,conn)
df

,industry,total_profit,profit_category
0,Retail,12233864.0,Alto
1,Entertainment and Media,11631745.0,Alto
2,Consulting,10470625.0,Alto
3,Technology,8758105.0,Alto
4,Finance,8602765.0,Alto
5,Law,4768601.0,Alto
6,Automotive,4619117.0,Alto
7,Manufacturing,4051281.0,Alto
8,Home Services,2675697.0,Alto
9,Hospitality,2454505.0,Alto


Evolución del Pronóstico y Beneficio Real: Análisis de la Trayectoria por
Categoría.

Enunciado: Calcula el pronóstico de beneficio para el año 2022 y el beneficio real para
el primer trimestre de 2020 y el tercer trimestre de 2021, agrupando los resultados por
categoría de producto. Además, queremos identificar la oportunidad más antigua y la
más reciente dentro de cada categoría.

In [32]:
#4.Evolución del Pronóstico y Beneficio Real: Análisis de la Trayectoria por Categoría

query="""
    SELECT
    COALESCE(combined_data.category, 'Desconocido') AS category,  -- Evita NULL en la categoría
    SUM(CASE WHEN combined_data.year = 2022 THEN combined_data.forecast ELSE 0 END) AS forecast_2022,
    SUM(CASE WHEN combined_data.year = 2020 AND combined_data.quarter_of_year = 'Q1' THEN combined_data.profit ELSE 0 END) AS profit_2020_q1,
    SUM(CASE WHEN combined_data.year = 2021 AND combined_data.quarter_of_year = 'Q3' THEN combined_data.profit ELSE 0 END) AS profit_2021_q3,
    MIN(combined_data.opportunity_age) AS oldest_opportunity,
    MAX(combined_data.opportunity_age) AS newest_opportunity
FROM (
    -- 🔹 PRIMER LEFT JOIN: Unimos sales con forecasts
    SELECT
        s.category AS category,  -- 👈 Nos aseguramos de que category esté bien referenciado
        s.year,
        s.quarter_of_year,
        s.profit,
        f.forecast,
        f.opportunity_age
    FROM sales AS s
    LEFT JOIN forecasts AS f ON s.category = f.category AND s.year = f.year
    WHERE (s.year IN (2020, 2021) AND s.quarter_of_year IN ('Q1', 'Q3'))
        OR f.year = 2022

    UNION

    -- 🔹 SEGUNDO LEFT JOIN: Unimos forecasts con sales
    SELECT
        f.category AS category,  -- 👈 Aquí también nos aseguramos de renombrarlo correctamente
        s.year,
        s.quarter_of_year,
        s.profit,
        f.forecast,
        f.opportunity_age
    FROM forecasts AS f
    LEFT JOIN sales AS s ON s.category = f.category AND s.year = f.year
    WHERE (s.year IN (2020, 2021) AND s.quarter_of_year IN ('Q1', 'Q3'))
        OR f.year = 2022
) AS combined_data  -- 🔹 Nos aseguramos de que la subconsulta tenga un alias válido
GROUP BY combined_data.category
ORDER BY combined_data.category;

"""

df=pd.read_sql_query(query,conn)
df

,category,forecast_2022,profit_2020_q1,profit_2021_q3,oldest_opportunity,newest_opportunity
0,Break room,0,2077967.0,5107911.0,2,378
1,Chairs,0,1148111.0,188800.0,3,173
2,Desks,0,1114668.0,1934225.0,3,383
3,Electronics,0,881830.0,1324853.0,3,383


Análisis completo de crecimiento regional en ventas y beneficios 2020-2021


In [33]:
#5.Caso práctico.Análisis completo de crecimiento regional en ventas y beneficios 2020-2021

query="""
    SELECT
  a.region,
  SUM(
    CASE
      WHEN s.year = 2020 THEN s.total
    END
  ) AS ventas_2020,
  SUM(
    CASE
      WHEN s.year = 2021 THEN s.total
    END
  ) AS ventas_2021,
  ROUND(
    (
      (
        SUM(
          CASE
            WHEN s.year = 2021 THEN s.total
          END
        ) / NULLIF(
          SUM(
            CASE
              WHEN s.year = 2020 THEN s.total
            END
          ),
          0
        )
      ) - 1
    ) * 100,
    2
  ) AS crecimiento_porcentual,
  SUM(
    CASE
      WHEN s.year = 2020 THEN s.profit
    END
  ) AS beneficio_2020,
  SUM(
    CASE
      WHEN s.year = 2021 THEN s.profit
    END
  ) AS beneficio_2021,
  ROUND(
    (
      (
        SUM(
          CASE
            WHEN s.year = 2021 THEN s.profit
          END
        ) / NULLIF(
          SUM(
            CASE
              WHEN s.year = 2020 THEN s.profit
            END
          ),
          0
        )
      ) - 1
    ) * 100,
    2
  ) AS crecimiento_beneficio,
  SUM(
    CASE
      WHEN s.year = 2020 THEN s.product
    END
  ) AS productos_2020
  /* Desglose de productos y servicios 2020 */,
  SUM(
    CASE
      WHEN s.year = 2020 THEN s.support
    END
  ) AS soporte_2020,
  SUM(
    CASE
      WHEN s.year = 2020 THEN s.maintenance
    END
  ) AS mantenimiento_2020,
  SUM(
    CASE
      WHEN s.year = 2020 THEN s.parts
    END
  ) AS partes_2020,
  SUM(
    CASE
      WHEN s.year = 2021 THEN s.product
    END
  ) AS productos_2021
  /* Desglose de productos y servicios 2021 */,
  SUM(
    CASE
      WHEN s.year = 2021 THEN s.support
    END
  ) AS soporte_2021,
  SUM(
    CASE
      WHEN s.year = 2021 THEN s.maintenance
    END
  ) AS mantenimiento_2021,
  SUM(
    CASE
      WHEN s.year = 2021 THEN s.parts
    END
  ) AS partes_2021
FROM
  sales AS s
  JOIN accounts AS a ON s."ACCOUNT" = a."ACCOUNT"   /* Une ventas con cuentas */
GROUP BY
  a.region
ORDER BY
  crecimiento_porcentual DESC;
"""

df=pd.read_sql_query(query,conn)
df

,REGION,ventas_2020,ventas_2021,crecimiento_porcentual,beneficio_2020,beneficio_2021,crecimiento_beneficio,productos_2020,soporte_2020,mantenimiento_2020,partes_2020,productos_2021,soporte_2021,mantenimiento_2021,partes_2021
0,NAM,24403809.0,44179349.0,81.03,8612572.0,15611889.0,81.27,17688750.0,1200000.0,4090659.0,1424400.0,30564369.0,1425000.0,7914980.0,4275000.0
1,EMEA,35375010.0,43827761.0,23.89,12200575.0,15036866.0,23.25,24906063.0,1500000.0,6108947.0,2860000.0,30130918.0,1915000.0,8416073.0,3365770.0
2,APAC,13362428.0,8584998.0,-35.75,4783214.0,3139730.0,-34.36,9519116.0,700000.0,2116608.0,1026705.0,4770431.0,500000.0,1899767.0,1414800.0
3,LATAM,NaN,4248702.0,NaN,NaN,1378697.0,NaN,NaN,NaN,NaN,NaN,2202284.0,600000.0,646323.0,800095.0
